In [10]:
# import pymysql
import pandas as pd
import mysql.connector
from mysql.connector import Error
from cryptography.fernet import Fernet

In [20]:
# 데이터를 암호화할 키를 생성합니다.
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# CSV 파일을 읽어옵니다.
df = pd.read_csv('/home/sh/lab/ragk/ChatbotData.csv')

# MySQL에 연결합니다.
try:
    connection = mysql.connector.connect(host='',
                                         database='',
                                         user='',
                                         password='')
    if connection.is_connected():
        cursor = connection.cursor()

        # 각 행에 대해 암호화하여 INSERT 문을 실행합니다.
        for i in range(len(df)):
            encrypted_Q = cipher_suite.encrypt(df['Q'][i].encode())
            encrypted_A = cipher_suite.encrypt(df['A'][i].encode())
            label = int(df['label'][i])

            insert_query = """INSERT INTO chatbot_table  (Q, A, label) VALUES (%s, %s, %s)"""
            record = (encrypted_Q, encrypted_A, label)
            cursor.execute(insert_query, record)

        connection.commit()

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

MySQL connection is closed


In [23]:
import mysql.connector

# MySQL에 연결합니다.
connection = mysql.connector.connect(host='',
                                         database='',
                                         user='',
                                         password='')
cursor = connection.cursor()

# 모든 테이블의 이름을 가져옵니다.
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

for table_name in tables:
    table_name = table_name[0]
    print(f"Table: {table_name}")
    
    # 각 테이블의 모든 데이터를 가져옵니다.
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    print()

# 연결을 닫습니다.
cursor.close()
connection.close()

Table: chatbot_table
(1, b'gAAAAABlauPk-JVjsHcaQJn2D4kfJoKGtnpG58yJ3M1NmWMxWp17OK0fPUqqbonJCCLXrlS96cXigs-hbIqhr7Fm-KngAIlJmw==', b'gAAAAABlauPkmPVA9yWmCLE0tWMwGUr4nrHu3dDfjm30n_xWH8FHPAxlmH9Rsq39y3ijL3XmCpdBwJ_y1mFYu0Jtl4cZXXuWbTXFUv4CHw0RtSKs_lvZghU=', 0)
(2, b'gAAAAABlauPklQCHSNnmkh-UsyYSIo9IiY8-5d1gYLjGCg6i8pbp2ezUuRb-zZOzmwfPtkWB1a7WTSCfGOmygemtp3v4R7gHX1SkI7w-lG4wuJHogOIrzlA=', b'gAAAAABlauPkSd5-x2a8ipwpOk1ibRzcLBgJamaE1FXz0m62JLRgjmLPTMLRUHHAoWNv4vDPWOm85cbgZNXxjqDv0nuQpWzUtBmoclAIxwi1ACP82ubp8XM=', 0)
(3, b'gAAAAABlauPkcHEZk-xE7zDB-7dglyE-U83Ndu45h1_mO30BqavMO_4xkVHYtVRqNvNR4LnBMW7AIRrEOaZXNJMyu1m8dnkbs079tFXvoQWyt4QBo1bFI0Q=', b'gAAAAABlauPkLdOLwbDxbFhJ9z3Z7n7Cl4axqRI6bHpJTfOwH1yWXFDmyKIqeuYXqehw48pFEdX4jxJLcXWHsxFp0YXAT0wGf2kOT0ExK31Fs2LRkB4kSvU=', 0)
(4, b'gAAAAABlauPkSDFIRDYfxhskiTRCmIkPjyeUW0mvMRuR1NzYvDNAXxUyXcbHMlxIDraYUjCUCvn89hF4qNLYSjDDMrgi7b6ZBVM-WuR3ne_7iRzIjHAnwB6AaLz6m8vv90rSg2oNn000', b'gAAAAABlauPkcna-rELqQSDbD3qIYhSLAaZI8LpSvT7A1fVSJNe4KRl8Btmr3ab7rpJ_LG8KcuIP6

In [28]:
# MySQL에 연결합니다.
connection = mysql.connector.connect(host='',
                                         database='',
                                         user='',
                                         password='')

cursor = connection.cursor()


# chatbot_table 테이블의 모든 데이터를 가져옵니다.
cursor.execute("SELECT * FROM chatbot_table")
rows = cursor.fetchall()

for row in rows:
    id, encrypted_Q, encrypted_A, label = row

    # 암호화된 Q, A를 복호화합니다.
    decrypted_Q = cipher_suite.decrypt(encrypted_Q).decode()
    decrypted_A = cipher_suite.decrypt(encrypted_A).decode()

    print(f"id: {id}, Q: {decrypted_Q}, A: {decrypted_A}, label: {label}")
    
    if id ==10:
        break
    

# 연결을 닫습니다.
cursor.close()
connection.close()

id: 1, Q: 12시 땡!, A: 하루가 또 가네요., label: 0
id: 2, Q: 1지망 학교 떨어졌어, A: 위로해 드립니다., label: 0
id: 3, Q: 3박4일 놀러가고 싶다, A: 여행은 언제나 좋죠., label: 0
id: 4, Q: 3박4일 정도 놀러가고 싶다, A: 여행은 언제나 좋죠., label: 0
id: 5, Q: PPL 심하네, A: 눈살이 찌푸려지죠., label: 0
id: 6, Q: SD카드 망가졌어, A: 다시 새로 사는 게 마음 편해요., label: 0
id: 7, Q: SD카드 안돼, A: 다시 새로 사는 게 마음 편해요., label: 0
id: 8, Q: SNS 맞팔 왜 안하지ㅠㅠ, A: 잘 모르고 있을 수도 있어요., label: 0
id: 9, Q: SNS 시간낭비인 거 아는데 매일 하는 중, A: 시간을 정하고 해보세요., label: 0
id: 10, Q: SNS 시간낭비인데 자꾸 보게됨, A: 시간을 정하고 해보세요., label: 0
